In [1]:
import warnings
warnings.filterwarnings('ignore')

# Onset 모델 평가

In [2]:
import glob
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model

def data_generator(cqt_paths, stft_paths, mel_paths, onset_paths, model):
    for cqt_path, stft_path, mel_path, onset_path in zip(cqt_paths, stft_paths, mel_paths, onset_paths):
        CQT = np.load(cqt_path)
        STFT = np.load(stft_path)
        MEL = np.load(mel_path)
        ONSET = np.load(onset_path)

        # 정규화
        CQT = CQT / np.max(CQT)
        STFT = STFT / np.max(STFT)
        MEL = MEL / np.max(MEL)

        model.reset_states()  # 한 곡마다 모델 상태 초기화
        for cqt, stft, mel, onset in zip(CQT, STFT, MEL, ONSET):
            yield ({'cqt_input': cqt, 'stft_input': stft, 'mel_input': mel}, onset)  # 한 세그먼트씩 반환

def evaluate(test_cqt, test_stft, test_mel, test_onset):
    test_cqt_paths, test_stft_paths, test_mel_paths, test_onset_paths = [], [], [], []

    for cqt_path, stft_path, mel_path, onset_path in zip(glob.glob(test_cqt), glob.glob(test_stft), glob.glob(test_mel), glob.glob(test_onset)):
        test_cqt_paths.append(cqt_path)
        test_stft_paths.append(stft_path)
        test_mel_paths.append(mel_path)
        test_onset_paths.append(onset_path)

    output_signature = (
        {
            'cqt_input': tf.TensorSpec(shape=(100, 264), dtype=tf.float32),
            'stft_input': tf.TensorSpec(shape=(100, 257), dtype=tf.float32),
            'mel_input': tf.TensorSpec(shape=(100, 128), dtype=tf.float32)
        },
        tf.TensorSpec(shape=(100, 88), dtype=tf.int8)
    )

    batch_size = 10

    model = load_model('models/onset_detector_v3.h5')

    test_set = tf.data.Dataset.from_generator(lambda: data_generator(test_cqt_paths, test_stft_paths, test_mel_paths, test_onset_paths, model), output_signature=output_signature)
    test_set = test_set.batch(batch_size, drop_remainder=True).prefetch(tf.data.experimental.AUTOTUNE)

    steps_per_epoch = sum([len(np.load(file)) for file in test_cqt_paths]) // batch_size

    loss, accuracy = model.evaluate(test_set, steps=steps_per_epoch, verbose=2)
    print(f'Test Loss: {loss}')
    print(f'Test Accuracy: {accuracy}')

if __name__ == '__main__':
    evaluate('data/preprocessed/testX/cqt/*.npy', 'data/preprocessed/testX/stft/*.npy', 'data/preprocessed/testX/mel/*.npy', 'data/preprocessed/testONSET/*.npy')

2024-07-21 20:43:31.096345: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-21 20:43:31.275296: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-21 20:43:31.275321: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-21 20:43:31.275776: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-21 20:43:31.342530: I tensorflow/core/platform/cpu_feature_g

INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA GeForce GTX 1660 SUPER, compute capability 7.5


2024-07-21 20:43:33.869938: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-21 20:43:33.885982: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-21 20:43:33.886027: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-21 20:43:33.886342: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-21 20:43:33.891682: I tensorflow/compile

2024-07-21 20:43:38.165724: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8700


570/570 - 15s - loss: 0.0184 - accuracy: 0.0712 - 15s/epoch - 26ms/step
Test Loss: 0.018394172191619873
Test Accuracy: 0.07119999825954437


# Offset 모델 평가

In [3]:
import glob
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model

def data_generator(cqt_paths, stft_paths, mel_paths, offset_paths, model):
    for cqt_path, stft_path, mel_path, offset_path in zip(cqt_paths, stft_paths, mel_paths, offset_paths):
        CQT = np.load(cqt_path)
        STFT = np.load(stft_path)
        MEL = np.load(mel_path)
        OFFSET = np.load(offset_path)

        # 정규화
        CQT = CQT / np.max(CQT)
        STFT = STFT / np.max(STFT)
        MEL = MEL / np.max(MEL)

        model.reset_states()  # 한 곡마다 모델 상태 초기화
        for cqt, stft, mel, offset in zip(CQT, STFT, MEL, OFFSET):
            yield ({'cqt_input': cqt, 'stft_input': stft, 'mel_input': mel}, offset)  # 한 세그먼트씩 반환

def evaluate(test_cqt, test_stft, test_mel, test_offset):
    test_cqt_paths, test_stft_paths, test_mel_paths, test_offset_paths = [], [], [], []

    for cqt_path, stft_path, mel_path, offset_path in zip(glob.glob(test_cqt), glob.glob(test_stft), glob.glob(test_mel), glob.glob(test_offset)):
        test_cqt_paths.append(cqt_path)
        test_stft_paths.append(stft_path)
        test_mel_paths.append(mel_path)
        test_offset_paths.append(offset_path)

    output_signature = (
        {
            'cqt_input': tf.TensorSpec(shape=(100, 264), dtype=tf.float32),
            'stft_input': tf.TensorSpec(shape=(100, 257), dtype=tf.float32),
            'mel_input': tf.TensorSpec(shape=(100, 128), dtype=tf.float32)
        },
        tf.TensorSpec(shape=(100, 88), dtype=tf.int8)
    )

    batch_size = 10

    model = load_model('models/offset_detector_v3.h5')

    test_set = tf.data.Dataset.from_generator(lambda: data_generator(test_cqt_paths, test_stft_paths, test_mel_paths, test_offset_paths, model), output_signature=output_signature)
    test_set = test_set.batch(batch_size, drop_remainder=True).prefetch(tf.data.experimental.AUTOTUNE)

    steps_per_epoch = sum([len(np.load(file)) for file in test_cqt_paths]) // batch_size

    loss, accuracy = model.evaluate(test_set, steps=steps_per_epoch, verbose=2)
    print(f'Test Loss: {loss}')
    print(f'Test Accuracy: {accuracy}')

if __name__ == '__main__':
    evaluate('data/preprocessed/testX/cqt/*.npy', 'data/preprocessed/testX/stft/*.npy', 'data/preprocessed/testX/mel/*.npy', 'data/preprocessed/testOFFSET/*.npy')

570/570 - 15s - loss: 0.0690 - accuracy: 0.1812 - 15s/epoch - 26ms/step
Test Loss: 0.06899569183588028
Test Accuracy: 0.18116316199302673
